# Gold Layer ETL
Time to ingest data into empry schemas once again. This time we are bringing the data from the silver layer into the gold layer. 

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as F

### Customer Profile Table
Which features of the customer base are the most relevant is maximizing expenditure?

In [0]:
%sql
INSERT INTO gold.customer_profile
SELECT
    c.education,
    c.marital_status,
    c.kidhome,
    c.teenhome,
    c.income,
    c.complain,

    -- Total Spending = MntWines + MntFruits + ... + MntSweetProds (Obs: *no* MntGoldProds, as its a different kind of category)  
    SUM(
        f.MntWines +
        f.MntFruits +
        f.MntMeatProducts +
        f.MntFishProducts +
        f.MntSweetProducts
    ) AS total_spending,

    -- Average Spending 
    AVG(
        f.MntWines +
        f.MntFruits +
        f.MntMeatProducts +
        f.MntFishProducts +
        f.MntSweetProducts
    ) AS avg_spent_per_customer

FROM silver.FACT_Sales f
JOIN silver.DIM_Customer c
  ON f.fk_customer = c.id_customer

GROUP BY
    c.education,
    c.marital_status,
    c.kidhome,
    c.teenhome,
    c.income,
    c.complain;


In [0]:
%sql
-- Test
SELECT * FROM gold.customer_profile LIMIT 15

In [0]:
%sql
SELECT
  education,
  ROUND(AVG(total_spending), 2) AS avg_spending
FROM gold.customer_profile
GROUP BY education
ORDER BY avg_spending DESC;

In [0]:
%sql
SELECT
  marital_status,
  ROUND(AVG(total_spending),2) AS avg_spending
FROM gold.customer_profile
GROUP BY marital_status
ORDER BY avg_spending DESC;


In [0]:
%sql
SELECT
  kidhome,
  ROUND(AVG(total_spending),2) AS avg_spending
FROM gold.customer_profile
GROUP BY kidhome
ORDER BY kidhome;


### Customer Spending Summary
No modeling was used, only joins.

In [0]:
%skip
%sql
SELECT
  education,
  marital_status,
  SUM(MntWines + MntFruits + MntMeatProducts + MntFishProducts + MntSweetProducts) AS total_spending,
  ROUND(
    AVG(
      MntWines + MntFruits + MntMeatProducts +
      MntFishProducts + MntSweetProducts
    ), 2
  ) AS avg_spending,
  COUNT(*) AS customers
FROM silver.fact_sales fs
JOIN silver.dim_customer dc
  ON fs.fk_customer = dc.id_customer
GROUP BY education, marital_status
ORDER BY avg_spending DESC;

In [0]:
%skip
%sql
WITH customer_spending AS (
    SELECT
        f.fk_customer,

        -- Total spending per customer
        SUM(
            f.MntWines +
            f.MntFruits +
            f.MntMeatProducts +
            f.MntFishProducts +
            f.MntSweetProducts
        ) AS total_spending

    FROM silver.FACT_Sales f
    GROUP BY f.fk_customer
)

SELECT
    c.education,
    c.marital_status,
    c.kidhome,
    c.teenhome,
    c.income,
    c.complain,

    COUNT(DISTINCT c.id_customer) AS total_customers,

    -- Métricas principais
    AVG(cs.total_spending) AS avg_spending_per_customer,
    SUM(cs.total_spending) AS total_spending_group

FROM customer_spending cs
JOIN silver.DIM_Customer c
  ON cs.fk_customer = c.id_customer

GROUP BY
    c.education,
    c.marital_status,
    c.kidhome,
    c.teenhome,
    c.income,
    c.complain
ORDER BY avg_spending_per_customer DESC;


In [0]:
%sql
SELECT
  dc.Education,
  dc.Marital_Status,
  SUM(
    fs.MntWines +
    fs.MntFruits +
    fs.MntMeatProducts +
    fs.MntFishProducts +
    fs.MntSweetProducts
  ) AS total_spending
FROM silver.fact_sales fs
JOIN silver.dim_customer dc
  ON fs.fk_customer = dc.id_customer
GROUP BY
  dc.Education,
  dc.Marital_Status

#C)

In [0]:
%sql
-- Purchase Channel
SELECT
  AVG(NumWebPurchases) AS avg_web,
  AVG(NumStorePurchases) AS avg_store,
  AVG(NumCatalogPurchases) AS avg_catalog
FROM silver.fact_sales;


# Extra

In [0]:
df_gold = spark.table("gold.customer_profile")

df_gold.select(
    F.corr("income", "total_spending").alias("corr_income_spending")
).display()


In [0]:
pdf = df_gold.select(
    "total_spending",
    "income",
    "kidhome",
    "teenhome"
).toPandas()

sns.heatmap(pdf.corr(), annot=True, cmap="coolwarm")
plt.show()

Evidently total spending is positively correlated with income (not as much as expected though), but it is intriguing that is is *negatively* correlated with having kids at home. Having a teen at home seems not to be relevant.